data source: http://help.sentiment140.com/for-students

learn bert NLP course https://www.udemy.com/course/bert-nlp-algorithm/learn/lecture/17347024#overview

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random


from google.colab import drive

In [2]:
  !pip install bert-for-tf2

     |████████████████████████████████| 51kB 6.5MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30537 sha256=a51045b03491db122b1188055df4b8b5b6d39d13dcac07b8e8b1c6f3a8686a1f
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7303 sha256=b04c30313f641aab46103bfcbbd2c4e357124e596df62503844a66d78ed83e56
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19474 sha256=4a57bda369c1bedb337ad28d155c353c2b78791c413b1be0717597107b191b53
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 22.6MB/s 


In [4]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [5]:
print(tf.version.VERSION)

2.4.0


In [6]:
drive.mount("/content/drive")

Mounted at /content/drive


## Read data and load to dataframe

In [56]:
columns = ["sentiment", "id", "date", "query", "user", "text"]

In [57]:
data = pd.read_csv("/content/drive/MyDrive/BERT/sentiment_data/train.csv",
                   header=None, names=columns, engine="python", encoding="latin1")

In [58]:
data.head(2)

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...


In [59]:
data.drop(["id", "date", "query", "user"], inplace=True, axis=1)

## DATA cleaning

In [60]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, "lxml").get_text()
  tweet = re.sub(r'@[A-Za-z0-9]+', ' ', tweet)
  tweet = re.sub(r'https?://[A-Za-z0-9./]+', ' ', tweet)
  # tweet = re.sub(r'[a-zA-Z\.?!\'"]+', ' ', tweet)
  tweet = re.sub(r'\s+', ' ', tweet)
  tweet = re.sub(r'^\s+', '', tweet)
  # lowercase everythin
  tweet = tweet.lower()
  return tweet

In [61]:
data["clean_text"] = data["text"].apply(clean_tweet)

In [62]:
data.head(2)

,sentiment,text,clean_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- awww, that's a bummer. you shoulda got david..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...


In [63]:
data.loc[(data.sentiment==4), "sentiment"] = 1

In [64]:
data[data["sentiment"]!=0].head(2)

,sentiment,text,clean_text
800000,1,I LOVE @Health4UandPets u guys r the best!!,i love u guys r the best!!
800001,1,im meeting up with one of my besties tonight! ...,im meeting up with one of my besties tonight! ...


## Tokenisation

In [65]:
FullTokeniser = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() 
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokeniser(vocab_file, do_lower_case)

we only use the first sentence for BERT inputs and so we add the CLS at the beginning and SEP at the end

In [66]:
def encode_sentence(sent):
  return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [67]:
data["data_inputs"] = data["clean_text"].apply(encode_sentence)

In [68]:
data.head(2)

,sentiment,text,clean_text,data_inputs
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- awww, that's a bummer. you shoulda got david...","[[CLS], -, aw, ##w, ##w, ,, that, ', s, a, bum..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...,"[[CLS], is, upset, that, he, can, ', t, update..."


## dataset creation

We need to create 3 different inputs for each sentence

In [69]:
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)


def get_mask(tokens):
  return np.char.not_equal(tokens, "[PAD]").astype(int)


def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == "[SEP]":
      current_seg_id = 1 - current_seg_id # toggle between 1 and 0
  return seg_ids

We will create padded batches (so we pad sentences for each batch independently). This way we add the minimum number of padded tokens possible. For that we sort sentences by length, add padded_batches and then shuffle

In [70]:
data["len"] = data["data_inputs"].str.len()

In [71]:
data.head(2)

,sentiment,text,clean_text,data_inputs,len
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- awww, that's a bummer. you shoulda got david...","[[CLS], -, aw, ##w, ##w, ,, that, ', s, a, bum...",29
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...,"[[CLS], is, upset, that, he, can, ', t, update...",31


In [72]:
data["get_ids"] = data["data_inputs"].apply(get_ids)
data["get_mask"] = data["data_inputs"].apply(get_mask)
data["get_segments"] = data["data_inputs"].apply(get_segments)

In [73]:
data["sorted_all"] = data[["get_ids", "get_mask", "get_segments", "sentiment"]]\
  .apply(lambda x: tuple([[x[0], x[1], x[2]], x[3]]), axis=1)

In [74]:
print(data["sorted_all"][0])

([[101, 1011, 22091, 2860, 2860, 1010, 2008, 1005, 1055, 1037, 26352, 5017, 1012, 2017, 2323, 2050, 2288, 2585, 12385, 1997, 2353, 2154, 2000, 2079, 2009, 1012, 1025, 1040, 102], array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1]), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 0)


In [75]:
# do a random shuffling
data = data.sample(frac=1).reset_index(drop=True)

In [76]:
# sort the values
data.sort_values(by=["len"], inplace=True)

In [77]:
data.head(2)

,sentiment,text,clean_text,data_inputs,len,get_ids,get_mask,get_segments,sorted_all
535729,0,@whinstonr,,"[[CLS], [SEP]]",2,"[101, 102]","[1, 1]","[0, 0]","([[101, 102], [1, 1], [0, 0]], 0)"
910981,1,@willbeblue,,"[[CLS], [SEP]]",2,"[101, 102]","[1, 1]","[0, 0]","([[101, 102], [1, 1], [0, 0]], 1)"


In [78]:
# we dont want the tweets that dont have data
data = data[data["len"]>7]

In [79]:
data.head(2)

,sentiment,text,clean_text,data_inputs,len,get_ids,get_mask,get_segments,sorted_all
273658,1,@geoann07 ur welcome gurl god bless,ur welcome gurl god bless,"[[CLS], ur, welcome, gu, ##rl, god, bless, [SEP]]",8,"[101, 24471, 6160, 19739, 12190, 2643, 19994, ...","[1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","([[101, 24471, 6160, 19739, 12190, 2643, 19994..."
837283,0,@colbylevi it's such a shame,it's such a shame,"[[CLS], it, ', s, such, a, shame, [SEP]]",8,"[101, 2009, 1005, 1055, 2107, 1037, 9467, 102]","[1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","([[101, 2009, 1005, 1055, 2107, 1037, 9467, 10..."


In [80]:
sorted_all = data.sorted_all.values

In [81]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [82]:
next(iter(all_dataset))

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[  101, 24471,  6160, 19739, 12190,  2643, 19994,   102],
        [    1,     1,     1,     1,     1,     1,     1,     1],
        [    0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [83]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [84]:
NB_BATCHES = math.ceil(len(sorted_all)/BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10  
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

## Model building

In [85]:
mysentence = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]

input_ids = get_ids(mysentence)
input_mask = get_mask(mysentence)
input_segments = get_segments(mysentence)

inputs = tf.stack(
    [
      tf.cast(input_ids, dtype=tf.int32),
      tf.cast(input_mask, dtype=tf.int32),
      tf.cast(input_segments, dtype=tf.int32)
    ], axis=0
)
inputs = tf.expand_dims(inputs, 0) # simulates a batch
print(inputs.shape)
bert_layer(inputs)

(1, 3, 6)


ValueError: ignored

In [86]:
class DCNNBERTEmbedding(tf.keras.Model):
  def __init__(self, nb_filters=50,
               FFN_units=512, nb_classes=2, dropout_rate=0.1, name="dcnn"):
    super(DCNNBERTEmbedding, self).__init__(name=name)
    
    # self.embedding = layers.Embedding(vocab_size, emb_dim)
    # instead of embedding we will use the bert layer
    self.bert_layer = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
        trainable=False)

    self.bigram = layers.Conv1D(filters=nb_filters, kernel_size=2,
                                padding="valid", activation="relu")
    self.trigram = layers.Conv1D(filters=nb_filters, kernel_size=3,
                                 padding="valid", activation="relu")
    self.fourgram = layers.Conv1D(filters=nb_filters, kernel_size=4,
                                  padding="valid", activation="relu")
    self.pool = layers.GlobalMaxPooling1D()
    self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
    self.dropout = layers.Dropout(rate=dropout_rate)
    if nb_classes == 2:
      self.last_dense = layers.Dense(units=1, activation="sigmoid")
    else:
      self.last_dense = layers.Dense(units=nb_classes, activation="softmax")

  def embed_with_bert(self, all_tokens):
    _, embs = self.bert_layer(
        [all_tokens[:, 0, :],
         all_tokens[:, 1, :],
         all_tokens[:, 2, :]]
    )
    return embs

  def call(self, inputs, training):
    x = self.embed_with_bert(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool(x_3)
    merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3*nb_filters)
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training)
    output = self.last_dense(merged)

    return output

## training ophase

In [87]:
NB_FILTER = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [88]:
dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTER,
            FFN_units=FFN_UNITS, nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [89]:
if NB_CLASSES == 2:
  dcnn.compile(loss="binary_crossentropy",
               optimizer="adam", metrics=["accuracy"])
else:
  dcnn.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
               metrics=["sparse_categorical_accuracy"])

In [90]:
checkpoint_path = "/content/drive/MyDrive/BERT/ckpt_bert_embedding"

ckpt = tf.train.Checkpoint(dcnn=dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("latest checkpoint restored!")

latest checkpoint restored!


In [91]:
class MyCustomCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    ckpt_manager.save()
    print("checkpoint saved at {}".format(checkpoint_path))

In [92]:
dcnn.fit(train_dataset, epochs=NB_EPOCHS, callbacks=[MyCustomCallback()])

Epoch 1/5
41039/41039 [==============================] - 2437s 59ms/step - loss: 0.3952 - accuracy: 0.8300
checkpoint saved at /content/drive/MyDrive/BERT/ckpt_bert_embedding
Epoch 2/5
41039/41039 [==============================] - 2414s 58ms/step - loss: 0.3495 - accuracy: 0.8491
checkpoint saved at /content/drive/MyDrive/BERT/ckpt_bert_embedding
Epoch 3/5
41039/41039 [==============================] - 2413s 58ms/step - loss: 0.3399 - accuracy: 0.8540
checkpoint saved at /content/drive/MyDrive/BERT/ckpt_bert_embedding
Epoch 4/5
41039/41039 [==============================] - 2414s 58ms/step - loss: 0.3335 - accuracy: 0.8570
checkpoint saved at /content/drive/MyDrive/BERT/ckpt_bert_embedding
Epoch 5/5
41039/41039 [==============================] - 2413s 58ms/step - loss: 0.3290 - accuracy: 0.8591
checkpoint saved at /content/drive/MyDrive/BERT/ckpt_bert_embedding


## model evaluation

In [93]:
results = dcnn.evaluate(test_dataset)
print(results)

4559/4559 [==============================] - 114s 25ms/step - loss: 0.3651 - accuracy: 0.8551
[0.36508601903915405, 0.8551217317581177]


In [94]:
def get_prediction(sentence):
  tokens = encode_sentence(sentence)

  input_ids = get_ids(tokens)
  input_mask = get_mask(tokens)
  input_segments = get_segments(tokens)

  inputs = tf.stack(
      [
       tf.cast(input_ids, dtype=tf.int32),
       tf.cast(input_mask, dtype=tf.int32),
       tf.cast(input_segments, dtype=tf.int32)
      ], axis=0
  )
  inputs = tf.expand_dims(inputs, 0) # simulates a batch

  output = dcnn(inputs, training=False)
  
  sentiment = math.floor(output * 2)

  if sentiment == 0:
    p = "positive"
  else:
    p = "negative"
  print("output of the model: {}".format(output))
  print("predicted sentiment: {}".format(p))

In [95]:
get_prediction("This movie was pretty interesting.")

output of the model: [[0.69493526]]
predicted sentiment: negative


In [96]:
get_prediction("This movie was shit")

output of the model: [[0.08882713]]
predicted sentiment: positive
